# Using LSTM and Doing More Exploration on the Bitcoin Dataset

In [4]:
# Improt necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [14]:
# Load Data and Extrapolate Datetime
file_path = '/Users/theni/Downloads/btcusd_1-min_data.csv'
data = pd.read_csv(file_path)
data['Timestamp'] = pd.to_datetime(data['Timestamp'], unit = 's')
data.set_index('Timestamp', inplace = True)

In [15]:
# Normalization
scaler = MinMaxScaler(feature_range = (0, 1))
data_scaled = scaler.fit_transform(data)

In [17]:
# Create Sequences
def create_seq(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i : i + seq_length].flatten())
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

In [19]:
# Set Sequence length
seq_length = 60 
X, y = create_seq(data_scaled, seq_length)

In [21]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [27]:
# Select a random subset of indices
subset_size = int(0.5 * X_train.shape[0])  # Adjust fraction as needed
indices = np.random.choice(X_train.shape[0], subset_size, replace=False)

# Create the sampled dataset
X_train_sampled = X_train[indices]
y_train_sampled = y_train[indices]


MemoryError: Unable to allocate 6.06 GiB for an array with shape (2712088, 300) and data type float64

In [26]:
# Train XGBoost Regression Model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model.fit(X_train_sampled, y_train_sampled)

NameError: name 'X_train_sampled' is not defined

In [ ]:
# Generate Predictions
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred))
print(f"RMSE: {rmse}"

In [ ]:
# Plot results
plt.figure(figsize=(12, 6))
plt.plot(y_test_actual, label="Actual Price")
plt.plot(y_pred, label="Predicted Price")
plt.legend()
plt.title("Bitcoin Price Prediction using XGBoost")
plt.show()